In [2]:
import tensorflow as tf
tf.__version__

'1.8.0'

In [1]:
import keras
from keras import Model, backend, Sequential
from keras.layers import Dense
from keras.activations import relu
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.models import save_model, load_model

Using TensorFlow backend.


In [2]:
NUM_OF_CLASS = 9
EPOCHS = 100
BATCH_SIZE = 16
LEARNING_RATE = 0.001

In [3]:
train_dir = './new_dataset/train'
val_dir = './new_dataset/val'

train_batches = ImageDataGenerator(preprocessing_function=keras.applications.mobilenet.preprocess_input).flow_from_directory(
    train_dir,
    target_size=(224, 224), 
    class_mode='categorical', 
    batch_size=BATCH_SIZE, 
    shuffle=True)
test_batches = ImageDataGenerator(preprocessing_function=keras.applications.mobilenet.preprocess_input).flow_from_directory(
    val_dir, 
    target_size=(224, 224), 
    class_mode='categorical', 
    batch_size=BATCH_SIZE, 
    shuffle=True)

Found 9063 images belonging to 9 classes.
Found 450 images belonging to 9 classes.


In [4]:
from keras.layers import Conv2D,MaxPool2D,GlobalAveragePooling2D

# Define network
mobilenet = keras.applications.mobilenet_v2.MobileNetV2(backend=keras.backend, layers=keras.layers, models=keras.models, utils=keras.utils)

x = mobilenet.layers[-2].output
x = Dense(128, activation='relu')(x)
predictions = Dense(NUM_OF_CLASS, activation='softmax')(x)

model = Model(inputs=mobilenet.input, outputs=predictions)
model.compile(SGD(lr=LEARNING_RATE), loss='categorical_crossentropy', metrics=['accuracy'])

14540800/14536120 [==============================] - 7s 1us/step


In [5]:
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.callbacks import LearningRateScheduler
import math

def step_decay(epoch):
    initial_lrate=0.1
    drop=0.5
    epochs_drop = 10.0
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate

lrate = LearningRateScheduler(step_decay)

EARLY_STOP_PARIENCE = 20

cb_early_stopper = EarlyStopping(monitor='val_loss', patience=EARLY_STOP_PARIENCE)
cb_checkpointer = ModelCheckpoint(filepath='./best_MobileNetV2.h5', monitor='val_loss',save_best_only=True, mode='auto')

history = model.fit_generator(train_batches, steps_per_epoch=10, validation_data=test_batches, validation_steps=10, epochs=EPOCHS, verbose=2, callbacks=[cb_checkpointer, cb_early_stopper, lrate])

Epoch 1/100
 - 14s - loss: 2.0653 - acc: 0.2938 - val_loss: 2.7172 - val_acc: 0.2062
Epoch 2/100
 - 5s - loss: 1.5997 - acc: 0.4688 - val_loss: 2.7739 - val_acc: 0.2625
Epoch 3/100
 - 6s - loss: 1.5473 - acc: 0.5125 - val_loss: 1.9040 - val_acc: 0.3493
Epoch 4/100
 - 5s - loss: 0.9414 - acc: 0.6625 - val_loss: 1.7171 - val_acc: 0.4062
Epoch 5/100
 - 5s - loss: 1.0417 - acc: 0.6562 - val_loss: 2.3686 - val_acc: 0.4625
Epoch 6/100
 - 6s - loss: 1.2309 - acc: 0.5875 - val_loss: 1.5880 - val_acc: 0.5000
Epoch 7/100
 - 6s - loss: 0.7849 - acc: 0.7250 - val_loss: 1.9933 - val_acc: 0.4125
Epoch 8/100
 - 5s - loss: 0.9723 - acc: 0.6937 - val_loss: 3.6603 - val_acc: 0.3187
Epoch 9/100
 - 5s - loss: 1.2153 - acc: 0.5750 - val_loss: 1.6620 - val_acc: 0.4726
Epoch 10/100
 - 5s - loss: 0.6242 - acc: 0.7875 - val_loss: 1.0858 - val_acc: 0.6375
Epoch 11/100
 - 5s - loss: 0.7062 - acc: 0.7750 - val_loss: 1.1413 - val_acc: 0.6500
Epoch 12/100
 - 6s - loss: 0.4761 - acc: 0.8500 - val_loss: 1.0608 - val_